In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import math
import linearmodels as lm
from tsmom_model3 import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:36: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:38: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:25: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:27: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:29: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=250,min_periods=100,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:29: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)

# Panel OLS regression

To see if the liquidity values are predictive I run the following regression

$$ r_{t+1}^i = a^i + b_t + cL_t^i + \varepsilon_{t+1}^i $$


where:
* $\alpha_i$ is entity effects
* $b_t$ is time effects
* $L_t^i$ is the liquidity measure for the $i$th asset

In [5]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')
    sector_rtns[sect]=cleansed[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:71: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:71: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


In [7]:
results_df=pd.DataFrame()
for sect in sector_zscores.keys():
    c_matrix=[]
    tstats=[]
    for t in [0,1]:
        for ent in [0,1]:
            data=pd.DataFrame(sector_zscores[sect].stack()) 
            px=pd.DataFrame(sector_rtns[sect].resample(rule='m',how='last').pct_change().shift().stack())
            ind=data.index
            px=px.loc[ind]
            res=lm.PanelOLS(px.to_panel()[0],
                    data.to_panel()[0], 
                    time_effects=t,
                    entity_effects=ent).fit(cov_type='clustered',cluster_time=True)
            c_matrix.append(res.params.Exog)
            tstats.append(res.tstats.Exog)
    results_df[sect]=pd.Series(c_matrix)*100
    results_df[sect + 'Tstats']=pd.Series(tstats)
results_df=results_df.set_index([['None','Ent','Time','Ent and Time']])

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively

In [8]:
results_df.T.round(2)

,None,Ent,Time,Ent and Time
Equities,-1.29,-1.69,-0.37,-0.55
EquitiesTstats,-8.61,-7.41,-3.75,-4.41
Commodities,-1.20,-1.35,-0.84,-1.08
CommoditiesTstats,-8.21,-6.66,-6.49,-7.26
Currencies,-0.35,-0.39,-0.13,-0.16
CurrenciesTstats,-2.46,-2.40,-0.51,-0.42
Fixed Income,-0.04,0.01,-0.04,-0.04
Fixed IncomeTstats,-1.02,0.26,-1.07,-1.10


In [13]:
print(results_df.T.round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  None &   Ent &  Time &  Ent and Time \\
\midrule
Equities           & -1.29 & -1.69 & -0.37 &         -0.55 \\
EquitiesTstats     & -8.61 & -7.41 & -3.75 &         -4.41 \\
Commodities        & -1.20 & -1.35 & -0.84 &         -1.08 \\
CommoditiesTstats  & -8.21 & -6.66 & -6.49 &         -7.26 \\
Currencies         & -0.35 & -0.39 & -0.13 &         -0.16 \\
CurrenciesTstats   & -2.46 & -2.40 & -0.51 &         -0.42 \\
Fixed Income       & -0.04 &  0.01 & -0.04 &         -0.04 \\
Fixed IncomeTstats & -1.02 &  0.26 & -1.07 &         -1.10 \\
\bottomrule
\end{tabular}



# FHT Annual portfolio sorts

In [9]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores_FHT={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores_FHT[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:53: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:54: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:55: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:55: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(

In [10]:
results_df_FHT=pd.DataFrame()
for sect in sector_zscores.keys():
    c_matrix=[]
    tstats=[]
    for t in [0,1]:
        for ent in [0,1]:
            data=pd.DataFrame(sector_zscores_FHT[sect].stack()) 
            px=pd.DataFrame(sector_rtns[sect].resample(rule='m',how='last').pct_change().shift().stack())
            ind=data.index
            px=px.loc[ind]
            res=lm.PanelOLS(px.to_panel()[0],
                    data.to_panel()[0], 
                    time_effects=t,
                    entity_effects=ent).fit(cov_type='clustered',cluster_time=True)
            c_matrix.append(res.params.Exog)
            tstats.append(res.tstats.Exog)
    results_df_FHT[sect]=pd.Series(c_matrix)*100
    results_df_FHT[sect + 'Tstats']=pd.Series(tstats)
results_df_FHT=results_df_FHT.set_index([['None','Ent','Time','Ent and Time']])

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively

In [11]:
results_df_FHT.T.round(2)

,None,Ent,Time,Ent and Time
Equities,-1.29,-1.29,-0.40,-0.36
EquitiesTstats,-5.62,-4.88,-2.47,-1.92
Commodities,-0.25,-0.31,-0.16,-0.16
CommoditiesTstats,-1.24,-1.56,-1.38,-1.39
Currencies,-0.21,-0.22,0.11,0.11
CurrenciesTstats,-1.45,-1.52,0.64,0.55
Fixed Income,0.00,0.01,0.01,-0.01
Fixed IncomeTstats,0.15,0.29,0.24,-0.41


In [15]:
print(results_df_FHT.T.round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  None &   Ent &  Time &  Ent and Time \\
\midrule
Equities           & -1.29 & -1.29 & -0.40 &         -0.36 \\
EquitiesTstats     & -5.62 & -4.88 & -2.47 &         -1.92 \\
Commodities        & -0.25 & -0.31 & -0.16 &         -0.16 \\
CommoditiesTstats  & -1.24 & -1.56 & -1.38 &         -1.39 \\
Currencies         & -0.21 & -0.22 &  0.11 &          0.11 \\
CurrenciesTstats   & -1.45 & -1.52 &  0.64 &          0.55 \\
Fixed Income       &  0.00 &  0.01 &  0.01 &         -0.01 \\
Fixed IncomeTstats &  0.15 &  0.29 &  0.24 &         -0.41 \\
\bottomrule
\end{tabular}

